## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-04-01-07-17 +0000,nyt,Judge Rules Trump Administration Illegally Can...,https://www.nytimes.com/2025/09/03/us/harvard-...
1,2025-09-04-01-02-27 +0000,nyt,"Venezuela, Drug Boats and Trump’s Latest Claim...",https://www.nytimes.com/2025/09/03/world/ameri...
2,2025-09-04-01-00-00 +0000,wsj,He Jokes About Trump and Invading Kenya—and Ma...,https://www.wsj.com/world/africa/uganda-presid...
3,2025-09-04-01-00-00 +0000,wsj,President Trump’s campaign to bring the Federa...,https://www.wsj.com/economy/central-banking/tr...
4,2025-09-04-01-00-00 +0000,wsj,Google Ruling Shows Antitrust Tools Struggle t...,https://www.wsj.com/us-news/law/google-search-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2309/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,45
168,china,31
270,putin,23
191,military,19
666,new,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
318,2025-09-03-05-07-00 +0000,nypost,"Trump accuses Putin, Kim Jong Un and Xi Jinpin...",https://nypost.com/2025/09/03/us-news/trump-ac...,208
186,2025-09-03-16-36-24 +0000,nypost,"Trump says Putin, Xi were just putting on a sh...",https://nypost.com/2025/09/03/us-news/trump-sa...,151
297,2025-09-03-08-26-24 +0000,nyt,"China’s Military Parade, in Photos: Xi Unveils...",https://www.nytimes.com/2025/09/03/world/asia/...,149
332,2025-09-03-02-57-51 +0000,nypost,China’s Xi hosts largest-ever military parade ...,https://nypost.com/2025/09/02/world-news/china...,149
335,2025-09-03-02-20-57 +0000,bbc,"Moment Xi, Putin and Kim pose with world leade...",https://www.bbc.com/news/videos/cly6ylk070zo?a...,136


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
318,208,2025-09-03-05-07-00 +0000,nypost,"Trump accuses Putin, Kim Jong Un and Xi Jinpin...",https://nypost.com/2025/09/03/us-news/trump-ac...
316,70,2025-09-03-05-23-07 +0000,nypost,Vice President JD Vance and second lady Usha w...,https://nypost.com/2025/09/03/us-news/vice-pre...
191,51,2025-09-03-16-30-19 +0000,nypost,"Top chefs Daniel Boulud, Jesus Duron cook up n...",https://nypost.com/2025/09/03/business/chefs-d...
28,47,2025-09-04-00-00-00 +0000,wsj,Hundreds of current and former Health and Huma...,https://www.wsj.com/politics/policy/rfk-jr-cdc...
3,47,2025-09-04-01-00-00 +0000,wsj,President Trump’s campaign to bring the Federa...,https://www.wsj.com/economy/central-banking/tr...
176,42,2025-09-03-17-16-35 +0000,nyt,"After Graham Linehan’s Arrest, Police Chief Sa...",https://www.nytimes.com/2025/09/03/world/europ...
90,40,2025-09-03-21-34-15 +0000,bbc,US strike on 'Venezuela drug boat': What do we...,https://www.bbc.com/news/articles/cdjzw3gplv7o...
68,36,2025-09-03-22-14-19 +0000,latimes,Alphabet's stock climbs after judge rules Goog...,https://www.latimes.com/business/story/2025-09...
74,35,2025-09-03-22-07-00 +0000,wsj,Florida is proposing to eliminate all vaccine ...,https://www.wsj.com/health/florida-seeks-to-en...
87,34,2025-09-03-21-38-28 +0000,nypost,Sen. Tom Cotton warns Biden-era Pentagon offic...,https://nypost.com/2025/09/03/us-news/pentagon...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
